#### Define the used fields from the UK biobank, and extract the patients diagnosed with vascular dementia

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from DataFields import DataFields
from DataFields import DateReportedFields
from UtilityFunctions import get_columns_from_chunk

UKBB_PATH = "~/biobank/ukb672220.csv"

# A dictionary containing the used features
datafields = DataFields

# Features of conditions specified by date
datereportedfields = DateReportedFields

In [3]:
# Filter by rows to extract patients with vascular dementia
def filter_by_vascular_dementia(chunk):
    col_1_filter = "Vascular Dementia Date First Reported"
    col_2_filter = "Date of vascular dementia report"
    col_combined = "Vascular Dementia Report Date"
    Y_col = "Has Vascular Dementia"
    
    # Filter rows where either col_1_filter or col_2_filter is not NaN
    filtered_chunk = chunk[chunk[col_1_filter].notna() | chunk[col_2_filter].notna()].copy()  # Make a copy here
    
    # Combine dates and add the new columns
    filtered_chunk.loc[:, col_combined] = pd.to_datetime(
        filtered_chunk[col_1_filter].combine_first(filtered_chunk[col_2_filter]), errors="coerce"
    )
    
    # Filter by dates to remove unspecified entries
    filtered_chunk.loc[:, Y_col] = (
        filtered_chunk[col_combined]
        .between(pd.Timestamp("1950-01-01"), pd.Timestamp("2030-12-31"))
        .astype(int)
    )

    return filtered_chunk


### Extract rows diagnosed with vascular dementia through the use of two fields

In [10]:
df = pd.read_csv("diagnosed.csv")

col_1_filter = "Vascular Dementia Date First Reported"
col_2_filter = "Date of vascular dementia report"
col_combined = "Vascular Dementia Report Date"

df = df.drop(columns=[col_1_filter, col_2_filter, col_combined])

df.to_csv("diagnosed.csv")


In [3]:

chunk_size = 10000

# Create an empty DataFrame to hold the filtered data
result_df = pd.DataFrame()

for chunk in pd.read_csv(UKBB_PATH, chunksize=chunk_size, low_memory=False):

    filtered_chunk = get_columns_from_chunk(chunk, datafields, oldest=True)

    filtered_chunk = filter_by_vascular_dementia(filtered_chunk)
    
    # Append to the result DataFrame
    result_df = pd.concat([result_df, filtered_chunk], ignore_index=True)

col_1_filter = "Vascular Dementia Date First Reported"
col_2_filter = "Date of vascular dementia report"
col_combined = "Vascular Dementia Report Date"

result_df = result_df.drop(columns=[col_1_filter, col_2_filter, col_combined])

# Save the result
result_df.to_csv('diagnosed.csv')

# Display the first 5 rows
result_df.head(5)

,id,Birth Year,Sex,Ethnicity,Education,Primary Hypertension,Secondary Hypertension,Vascular Dementia Date First Reported,Date of vascular dementia report,BMI Impedance,...,Red blood cell (erythrocyte) count,Red blood cell (erythrocyte) distribution width,Reticulocyte count,Reticulocyte percentage,White blood cell (leukocyte) count,Blood Pressure Diastolic,Blood Pressure Systolic,Pulse Rate at Blood Pressure,Vascular Dementia Report Date,Has Vascular Dementia
0,1002012,1940.0,1,1001.0,1.0,2009-08-13,NaN,2019-09-06,2019-09-06,27.0801,...,4.799,13.07,0.050,1.047,7.79,75.0,182.0,62.0,2019-09-06,1
1,1002324,1938.0,1,1001.0,-7.0,2000-10-01,NaN,2022-07-01,NaN,33.7984,...,4.080,15.80,0.134,3.280,3.60,84.0,146.0,73.0,2022-07-01,1
2,1005163,1941.0,0,1001.0,1.0,NaN,NaN,2022-09-01,NaN,30.3880,...,4.330,15.00,0.058,1.330,6.00,94.0,143.0,87.0,2022-09-01,1
3,1009462,1938.0,1,1001.0,-7.0,2004-01-01,NaN,2021-02-04,2021-02-04,32.2385,...,4.800,15.50,0.084,1.750,8.20,77.0,141.0,79.0,2021-02-04,1
4,1010028,1947.0,0,1001.0,-7.0,NaN,NaN,2019-04-11,2019-04-11,32.0399,...,4.440,13.40,0.042,0.950,7.10,71.0,131.0,55.0,2019-04-11,1
